**The TensorFlow notebooks for this project are still in development so may contain bugs and are lacking in documentation. Please see the PyStan notebooks for a clearer walkthrough.**

In [41]:
import time

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp

In [42]:
tfd = tfp.distributions

In [43]:
# Confirm GPU in use
tf.config.list_physical_devices('GPU')

[]

In [44]:
# Parameters
P = 10
N = 10 ** 2
SEED = 1729

In [45]:
# Ground truth
np.random.seed(SEED)
true_transmission_rate = np.random.beta(2, 10, P)
true_occurrence_rate = np.random.beta(2, 10, P)
base_rate = np.random.beta(2, 10, 1)

test_sensitivity = np.random.beta(4, 3, 1)  # True positive rate
test_specificity = np.random.beta(50, 2, 1)  # True negative rate
true_lambda = np.hstack([test_sensitivity, test_specificity])

In [46]:
# Simulate data
data = {}
for p in range(P):
    occurrence = np.random.binomial(1, true_occurrence_rate[p], N)
    transmission = occurrence * np.random.binomial(1, true_transmission_rate[p], N)
    data[f'O{p+1}'] = occurrence
    data[f'T{p+1}'] = transmission
data['T0'] = np.random.binomial(1, base_rate, N)
X = pd.DataFrame(data)

z = X.loc[:, X.columns.str.startswith('T')].sum(axis=1)
X = X.loc[:, X.columns.str.startswith('O')]
y = (z > 0).astype(int)

# Introducing false positives and negatives
y = y * np.random.binomial(1, true_lambda[0], N) + \
    (1 - y) * np.random.binomial(1, (1 - true_lambda[1]), N)

In [47]:
# Convert to tensors
X = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)
# Move to GPU
X = X + tf.fill(X.shape, 0.0)
y = y + tf.fill(y.shape, 0.0)

In [48]:
# Set antigen test mean and std. error for TP and TN rates (for strong priors)

# True positive
mean_tp = 0.73000
se_tp = 0.04133

# True negative
mean_tn = 0.99680
se_tn = 0.00066

mean_rates = [mean_tp, mean_tn]
se_rates = [se_tp, se_tn]
alphas = []
betas = []

for i in range(2):
    alphas.append((((1 - mean_rates[i])/se_rates[i]**2)-(1/mean_rates[i])) * (mean_rates[i]**2))
    betas.append(alphas[i]*((1/mean_rates[i])-1))
    
print('Shape parameters for test accuracy priors:')    
print('Alphas: ', alphas)
print('Betas: ', betas)

lambda_prior_params = np.array([alphas, betas]).T

Shape parameters for test accuracy priors:
Alphas:  [83.50230278926166, 7298.251978696008]
Betas:  [30.88441336041184, 23.429380348943674]


In [49]:
# Define log-likelihood
@tf.function
def censored_poisbinom_loglike(theta, rho, lambda_):
    target = 0
    # Pre-computation
    log1m_theta = tf.math.log(1-theta)
    log_lambda = tf.math.log(lambda_)
    log1m_lambda = tf.math.log(1-lambda_)
    # Support
    if tf.math.reduce_any(tf.math.logical_or(theta <= 0., theta >= 1.)):
        return -np.inf
    if tf.math.logical_or(rho <= 0., rho >= 1.):
        return -np.inf
    if tf.math.reduce_any(lambda_ <= 0.):
        return -np.inf
    # Priors (beta)
    target += (lambda_prior_params[0,0] - 1) * log_lambda[0] + \
              (lambda_prior_params[0,1] - 1) * log1m_lambda[0]
    target += (lambda_prior_params[1,0] - 1) * log_lambda[1] + \
              (lambda_prior_params[1,1] - 1) * log1m_lambda[1]
    # Likelihood
    s = tf.einsum('ij,j->i', X, log1m_theta) + tf.math.log(1-rho)
    target += tf.math.reduce_sum(tf.where(
        y == 1,
        tfp.math.log_add_exp(
            tfp.math.log1mexp(s) + log_lambda[0],
            s + log1m_lambda[1]
        ), 
        tfp.math.log_add_exp(
            s + log_lambda[1],
            tfp.math.log1mexp(s) + log1m_lambda[0]
        )
    ))
    return target

In [50]:
# Define negative log-likelihood and use AD to compute gradients
@tf.function
def censored_poisbinom_negloglike(params):
    theta, rho, lambda_ = tf.split(params, [P, 1, 2], axis=0)
    # need to take these back down to vectors and scalars:
    theta = tf.reshape(theta,(P,))
    rho = tf.reshape(rho,())
    lambda_ = tf.reshape(lambda_, (2,))
    return -1 * censored_poisbinom_loglike(theta, rho, lambda_)

@tf.function
def censored_poisbinom_negloglike_and_grad(params):
    return tfp.math.value_and_gradient(
        censored_poisbinom_negloglike, 
        params
    )

In [65]:
# Approximate MLE using gradient descent
start = tf.fill(P + 3, 0.5)

optim_results = tfp.optimizer.bfgs_minimize(
    censored_poisbinom_negloglike_and_grad, start, tolerance=1e-8
)

est_params = optim_results.position.numpy()
est_serr = np.sqrt(np.diagonal(optim_results.inverse_hessian_estimate.numpy()))

In [54]:
# Set model parameters
nuts_samples = 1000
nuts_burnin = 200
init_step_size=.3
init = [est_params[:P], est_params[P], est_params[P+1:]]

In [62]:
# Fit model
@tf.function
def nuts_sampler(init):
    nuts_kernel = tfp.mcmc.NoUTurnSampler(
        target_log_prob_fn=censored_poisbinom_loglike, 
        step_size=init_step_size,
    )
    adapt_nuts_kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(
        inner_kernel=nuts_kernel,
        num_adaptation_steps=nuts_burnin,
        step_size_getter_fn=lambda pkr: pkr.step_size,
        log_accept_prob_getter_fn=lambda pkr: pkr.log_accept_ratio,
        step_size_setter_fn=lambda pkr, new_step_size: pkr._replace(step_size=new_step_size)
    )

    samples = tfp.mcmc.sample_chain(
        num_results=nuts_samples,
        current_state=init,
        kernel=adapt_nuts_kernel,
        num_burnin_steps=nuts_burnin,
        parallel_iterations=10,
        trace_fn=None
    )
    return samples

start = time.time()
samples = nuts_sampler(init)
print(f"{time.time() - start:.02f} seconds elapsed")

73.33 seconds elapsed
